In [2]:
!pip install openpyxl
import numpy as np
import pandas as pd
import os
import re
import itertools

In [3]:
ns_st = ['泉中央', '八乙女', '黒松', '旭ヶ丘', '台原', '北仙台', '北四番丁', '勾当台公園', '広瀬通',
    '仙台', '五橋', '愛宕橋', '河原町', '長町一丁目', '長町', '長町南', '富沢']
ew_st = ['八木山動物公園', '青葉山', '川内', '国際センター', '大町西公園', '青葉通一番町', '仙台', '宮城野通', '連坊',
         '薬師堂', '卸町', '六丁の目', '荒井']
all_st = ns_st + [s for s in ew_st if s !='仙台']
cols = ['年', '月', '乗車駅', '降車駅', '合計', '定期券外', '普通券計', '普通券大', '乗継計', '乗継大', '乗継小', '地下売', 'バス売', '一日券', 'プリペ', 'ICSF', 'その他', '定期計', '通勤', '通学', '学都', '公用', '敬老', 'ふれあい', 'その他.1']

In [37]:
FY = 2018
df18 = pd.read_csv("data/Sendai_subway_OD_FY{}.csv".format(FY))
FY = 2019
df19 = pd.read_csv("data/Sendai_subway_OD_FY{}.csv".format(FY))
df19

,年,月,乗車駅,降車駅,合計,定期券外,普通券計,普通券大,乗継計,乗継大,...,ICSF,その他,定期計,通勤,通学,学都,公用,敬老,ふれあい,その他.1
0,2019,4,泉中央,八乙女,22407,11055,2040.0,1757.0,NaN,NaN,...,8768,NaN,11352,5677.0,232.0,1593.0,NaN,2624.0,1226.0,NaN
1,2019,4,泉中央,黒松,14636,7725,1415.0,1240.0,NaN,NaN,...,6176,NaN,6911,2849.0,258.0,976.0,NaN,2252.0,576.0,NaN
2,2019,4,泉中央,旭ヶ丘,20502,9040,1580.0,1312.0,NaN,NaN,...,7312,NaN,11462,4551.0,188.0,2043.0,NaN,3236.0,1444.0,NaN
3,2019,4,泉中央,台原,17746,7904,1392.0,1237.0,NaN,NaN,...,6418,1.0,9842,3755.0,262.0,3342.0,1.0,1714.0,768.0,NaN
4,2019,4,泉中央,北仙台,39411,17598,2768.0,2508.0,NaN,NaN,...,14674,1.0,21813,8801.0,2149.0,7857.0,1.0,1961.0,1044.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9767,2020,3,荒井,宮城野通,6091,2427,243.0,225.0,NaN,NaN,...,2130,NaN,3664,3020.0,NaN,178.0,NaN,169.0,297.0,NaN
9768,2020,3,荒井,連坊,2107,1019,120.0,109.0,NaN,NaN,...,876,NaN,1088,766.0,NaN,125.0,NaN,115.0,82.0,NaN
9769,2020,3,荒井,薬師堂,2040,840,130.0,118.0,NaN,NaN,...,691,NaN,1200,809.0,1.0,89.0,1.0,113.0,187.0,NaN
9770,2020,3,荒井,卸町,3507,1764,182.0,161.0,NaN,NaN,...,1545,NaN,1743,1262.0,1.0,170.0,NaN,163.0,147.0,NaN


In [20]:
df18.set_index(["月","乗車駅","降車駅"])

年     合計   定期券外    普通券計    普通券大  乗継計  乗継大  乗継小  地下売  バス売  ...  \
月 乗車駅 降車駅                                                                ...   
4 泉中央 八乙女   2018  20836   9902  2114.0  1874.0  NaN  NaN  NaN  NaN  NaN  ...   
      黒松    2018  13794   7062  1332.0  1142.0  NaN  NaN  NaN  NaN  NaN  ...   
      旭ヶ丘   2018  19562   8887  1626.0  1364.0  NaN  NaN  NaN  NaN  NaN  ...   
      台原    2018  17599   8033  1519.0  1306.0  NaN  NaN  NaN  NaN  NaN  ...   
      北仙台   2018  39399  17853  3136.0  2931.0  NaN  NaN  NaN  NaN  NaN  ...   
...          ...    ...    ...     ...     ...  ...  ...  ...  ...  ...  ...   
3 荒井  宮城野通  2019   6523   3007   459.0   421.0  NaN  NaN  NaN  NaN  NaN  ...   
      連坊    2019   2095    922   158.0   152.0  NaN  NaN  NaN  NaN  NaN  ...   
      薬師堂   2019   2556   1404   204.0   181.0  NaN  NaN  NaN  NaN  NaN  ...   
      卸町    2019   4009   2165   373.0   331.0  NaN  NaN  NaN  NaN  NaN  ...   
      六丁の目  2019   1056    615   136.0   108.0  NaN  NaN  NaN  NaN  NaN  ...   

             ICSF  その他    定期計      通勤      通学      学都   公用      敬老    ふれあい  \
月 乗車駅 降車駅                                                                    
4 泉中央 八乙女    7510  NaN  10934  5770.0   234.0  1439.0  5.0  2433.0  1053.0   
      黒松     5580  NaN   6732  2758.0   242.0   912.0  NaN  2182.0   638.0   
      旭ヶ丘    7113  NaN  10675  4092.0   222.0  1856.0  2.0  3238.0  1265.0   
      台原     6410  NaN   9566  3503.0   284.0  3838.0  2.0  1291.0   648.0   
      北仙台   14544  NaN  21546  8517.0  2577.0  7472.0  6.0  1864.0  1110.0   
...           ...  ...    ...     ...     ...     ...  ...     ...     ...   
3 荒井  宮城野通   2500  NaN   3516  2690.0     1.0   367.0  NaN   245.0   213.0   
      連坊      747  NaN   1173   681.0     NaN   264.0  NaN   141.0    87.0   
      薬師堂     923  NaN   1152   541.0     NaN   217.0  NaN   202.0   192.0   
      卸町     1700  NaN   1844  1205.0     NaN   237.0  2.0   218.0   182.0   
      六丁の目    424  NaN    441   231.0     NaN    77.0  5.0    80.0    48.0   

            その他.1  
月 乗車駅 降車駅          
4 泉中央 八乙女     NaN  
      黒松      NaN  
      旭ヶ丘     NaN  
      台原      NaN  
      北仙台     NaN  
...           ...  
3 荒井  宮城野通    NaN  
      連坊      NaN  
      薬師堂     NaN  
      卸町      NaN  
      六丁の目    NaN  

[9744 rows x 22 columns]

In [27]:
for m in range(1,13):
    mdf18 = df18.set_index(["月","乗車駅","降車駅"]).xs(m, level="月")
    mdf19 = df19.set_index(["月","乗車駅","降車駅"]).xs(m, level="月")
    print(m, mdf18.shape, mdf19.shape)

1 (812, 22) (812, 22)
2 (812, 22) (812, 22)
3 (812, 22) (812, 22)
4 (812, 22) (812, 22)
5 (812, 22) (812, 22)
6 (812, 22) (812, 22)
7 (812, 22) (812, 22)
8 (812, 22) (812, 22)
9 (812, 22) (812, 22)
10 (812, 22) (840, 22)
11 (812, 22) (812, 22)
12 (812, 22) (812, 22)


In [28]:
df18.set_index(["月","乗車駅","降車駅"]).xs(10, level="月")

年     合計   定期券外    普通券計    普通券大  乗継計  乗継大  乗継小  地下売  バス売  ...  \
乗車駅 降車駅                                                                ...   
泉中央 八乙女   2018  22401  10441  1929.0  1658.0  NaN  NaN  NaN  NaN  NaN  ...   
    黒松    2018  14757   7309  1262.0  1122.0  NaN  NaN  NaN  NaN  NaN  ...   
    旭ヶ丘   2018  23754  10922  2101.0  1425.0  NaN  NaN  NaN  NaN  NaN  ...   
    台原    2018  19699   7927  1455.0  1296.0  NaN  NaN  NaN  NaN  NaN  ...   
    北仙台   2018  42097  17812  2861.0  2643.0  NaN  NaN  NaN  NaN  NaN  ...   
...        ...    ...    ...     ...     ...  ...  ...  ...  ...  ...  ...   
荒井  宮城野通  2018   7015   3079   497.0   464.0  NaN  NaN  NaN  NaN  NaN  ...   
    連坊    2018   2506   1082   198.0   185.0  NaN  NaN  NaN  NaN  NaN  ...   
    薬師堂   2018   3045   1414   359.0   324.0  NaN  NaN  NaN  NaN  NaN  ...   
    卸町    2018   4314   2283   485.0   411.0  NaN  NaN  NaN  NaN  NaN  ...   
    六丁の目  2018   1204    736   170.0   148.0  NaN  NaN  NaN  NaN  NaN  ...   

           ICSF  その他    定期計      通勤      通学      学都   公用      敬老    ふれあい  \
乗車駅 降車駅                                                                    
泉中央 八乙女    7943  NaN  11960  5758.0   232.0  1723.0  1.0  2920.0  1326.0   
    黒松     5871  NaN   7448  2938.0   309.0  1157.0  1.0  2346.0   697.0   
    旭ヶ丘    7499  NaN  12832  4589.0   232.0  2308.0  NaN  4166.0  1537.0   
    台原     6384  NaN  11772  4142.0   354.0  4750.0  2.0  1808.0   716.0   
    北仙台   14777  NaN  24285  9219.0  2857.0  8894.0  4.0  2190.0  1121.0   
...         ...  ...    ...     ...     ...     ...  ...     ...     ...   
荒井  宮城野通   2515  NaN   3936  2690.0     4.0   662.0  NaN   309.0   271.0   
    連坊      874  NaN   1424   700.0     NaN   499.0  NaN   134.0    91.0   
    薬師堂    1010  NaN   1631   662.0     1.0   600.0  NaN   190.0   178.0   
    卸町     1698  NaN   2031  1281.0     5.0   268.0  NaN   318.0   159.0   
    六丁の目    430  NaN    468   224.0     1.0   112.0  4.0    76.0    51.0   

          その他.1  
乗車駅 降車駅          
泉中央 八乙女     NaN  
    黒松      NaN  
    旭ヶ丘     NaN  
    台原      NaN  
    北仙台     NaN  
...         ...  
荒井  宮城野通    NaN  
    連坊      NaN  
    薬師堂     NaN  
    卸町      NaN  
    六丁の目    NaN  

[812 rows x 22 columns]

In [47]:
Oct19df = df19[df19["月"]==10]
for st in all_st:
    print(st, len(Oct19df[Oct19df["乗車駅"]==st]["降車駅"]))

泉中央 28
八乙女 28
黒松 28
旭ヶ丘 28
台原 28
北仙台 28
北四番丁 28
勾当台公園 28
広瀬通 28
仙台 28
五橋 28
愛宕橋 28
河原町 28
長町一丁目 28
長町 28
長町南 28
富沢 28
八木山動物公園 28
青葉山 56
川内 28
国際センター 28
大町西公園 28
青葉通一番町 28
宮城野通 28
連坊 28
薬師堂 28
卸町 28
六丁の目 28
荒井 28


In [49]:
Oct19df[Oct19df["乗車駅"]=="青葉山"]

,年,月,乗車駅,降車駅,合計,定期券外,普通券計,普通券大,乗継計,乗継大,...,ICSF,その他,定期計,通勤,通学,学都,公用,敬老,ふれあい,その他.1
5376,2019,10,青葉山,泉中央,3193,675,61.0,56.0,NaN,NaN,...,581,NaN,2518,555.0,99.0,1793.0,NaN,28.0,43.0,NaN
5377,2019,10,青葉山,泉中央,563,108,11.0,11.0,NaN,NaN,...,96,NaN,455,98.0,14.0,333.0,NaN,1.0,9.0,NaN
5378,2019,10,青葉山,八乙女,704,125,9.0,8.0,NaN,NaN,...,111,NaN,579,295.0,NaN,273.0,NaN,10.0,1.0,NaN
5379,2019,10,青葉山,八乙女,126,18,NaN,NaN,NaN,NaN,...,18,NaN,108,59.0,NaN,46.0,NaN,3.0,NaN,NaN
5380,2019,10,青葉山,黒松,452,109,4.0,4.0,NaN,NaN,...,105,NaN,343,123.0,NaN,215.0,NaN,5.0,NaN,NaN
5381,2019,10,青葉山,黒松,81,19,1.0,1.0,NaN,NaN,...,18,NaN,62,19.0,NaN,42.0,NaN,1.0,NaN,NaN
5382,2019,10,青葉山,旭ヶ丘,615,129,13.0,13.0,NaN,NaN,...,113,NaN,486,145.0,NaN,327.0,NaN,14.0,NaN,NaN
5383,2019,10,青葉山,旭ヶ丘,125,31,5.0,5.0,NaN,NaN,...,25,NaN,94,28.0,NaN,64.0,NaN,2.0,NaN,NaN
5384,2019,10,青葉山,台原,675,175,15.0,11.0,NaN,NaN,...,158,NaN,500,202.0,10.0,252.0,NaN,18.0,18.0,NaN
5385,2019,10,青葉山,台原,109,19,2.0,2.0,NaN,NaN,...,17,NaN,90,39.0,2.0,43.0,NaN,1.0,5.0,NaN
